<a href="https://colab.research.google.com/github/HedersonSantos/SmartHome/blob/main/automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install fast_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 580 kB/s 


In [ ]:
!pip install evalml==0.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.2 MB 9.7 MB/s 
     |████████████████████████████████| 2.0 MB 32.2 MB/s 
     |████████████████████████████████| 354 kB 54.1 MB/s 
     |████████████████████████████████| 6.7 MB 30.7 MB/s 
     |████████████████████████████████| 281 kB 54.0 MB/s 
     |████████████████████████████████| 1.5 MB 33.4 MB/s 
     |████████████████████████████████| 11.2 MB 38.2 MB/s 
     |████████████████████████████████| 79.9 MB 96 kB/s 
     |████████████████████████████████| 9.8 MB 52.3 MB/s 
     |████████████████████████████████| 128 kB 34.6 MB/s 
     |████████████████████████████████| 69 kB 7.2 MB/s 
     |████████████████████████████████| 100 kB 10.2 MB/s 
     |████████████████████████████████| 46 kB 4.4 MB/s 
     |████████████████████████████████| 564 kB 42.3 MB/s 
     |██████████████████████████▊     | 37.3 MB 1.2 MB/s eta 0:00:07

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline
import seaborn as sns
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error
from fast_ml.feature_engineering import FeatureEngineering_DateTime


In [6]:
!wget https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/train.csv
!wget https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/test.csv
!wget https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/sample_submission.csv

--2022-06-02 17:43:16--  https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 360467 (352K) [text/plain]
Saving to: ‘train.csv.1’

train.csv.1         100%[===================>] 352.02K  --.-KB/s    in 0.02s   

2022-06-02 17:43:16 (15.3 MB/s) - ‘train.csv.1’ saved [360467/360467]

--2022-06-02 17:43:16--  https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170613 (167K) [text/plain]
Saving to: 

In [7]:
def plotaSeries(treino, teste, predito):
  if type(treino) == pd.core.series.Series:
    treino.plot(legend=True,label='Treino')
  if type(teste) == pd.core.series.Series:
    teste.plot(legend=True,label='Teste',figsize=(16,8))
  if  type(predito)==pd.core.series.Series:
    predito.plot(legend=True) 
  else:
    for p in predito.columns:
      predito[p].plot(legend=True)

In [9]:
df_treino = pd.read_csv('train.csv')
df_teste = pd.read_csv('test.csv')


In [10]:
dt_fe = FeatureEngineering_DateTime()

def features_eng(data):
    # Step 1
    data['Date']=data['Date'].copy() + ' ' + data['Time']
    data['Date']=pd.to_datetime(data['Date'])
    data=data.drop(['Time'],axis=1)
    
    # Step 2
    dt_fe.fit(data, datetime_variables=['Date'])
    data = dt_fe.transform(data)
    
    return data

In [11]:
train = features_eng(df_treino)
test = features_eng(df_teste)

/usr/local/lib/python3.7/dist-packages/fast_ml/feature_engineering.py:382: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df[pfx+f] = getattr(df[var].dt, f)


In [18]:
nunique_train=train.nunique().reset_index()
remove_col=nunique_train[(nunique_train[0]==len(train)) | (nunique_train[0]==0) | (nunique_train[0]==1) ]['index'].tolist()
remove_col

['Id',
 'Date',
 'Date:year',
 'Date:is_month_start',
 'Date:is_quarter_start',
 'Date:is_year_end',
 'Date:is_year_start',
 'Date:second']

In [20]:
def data_clean(data):
    # Step 1
    data=data.drop(remove_col,axis=1)
    
    # Step 2 (Data preprocessing)
    data['Date:is_weekend']=data['Date:is_weekend'].replace({False: 0, True: 1})
    data['Date:is_quarter_end']=data['Date:is_quarter_end'].replace({False: 0, True: 1})
    data['Date:is_month_end']=data['Date:is_month_end'].replace({False: 0, True: 1})
    
    # Step 3
    data=data.drop(['Date:time'],axis=1)
    
    # Step 4
    data['Date:day_part'] = data['Date:day_part'].fillna(value=np.nan)
    
    # Step 5
    data['Date:day_part'] = data['Date:day_part'].fillna(value='midnight')
    
    # Step 6
    data['Date:day_part'] = data['Date:day_part'].replace({'midnight':0, 'dawn':1, 'early morning':2, 
                                                               'late morning':3, 'noon':4, 'afternoon':5,
                                                               'evening':6, 'night':7})
    
    return data

In [21]:
train = data_clean(train)
test = data_clean(test)

In [23]:
import re

train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

train.head()

,CO2_dinningroom,CO2_room,Relative_humidity_dinningroom,Relative_humidity_room,Lighting_dinningroom,Lighting_room,Meteo_Rain,Meteo_Sun_dusk,Meteo_Wind,Meteo_Sun_light_in_west_facade,Meteo_Sun_light_in_east_facade,Meteo_Sun_light_in_south_facade,Meteo_Sun_irradiance,Outdoor_relative_humidity_Sensor,Day_of_the_week,Indoor_temperature_room,Datequarter,Datemonth,Dateday,Dateday_of_week,Dateday_of_year,Dateweekofyear,Dateis_month_end,Dateis_quarter_end,Datehour,Dateminute,Dateis_weekend,Dateday_part
0,216.560,221.920,39.9125,42.4150,81.6650,113.520,0.0,623.360,1.42625,9690.24,12604.20,95436.8,758.880,48.3750,2.0,17.8275,1,3,13,1,73,11,0,0,11,45,0,4
1,219.947,220.363,39.9267,42.2453,81.7413,113.605,0.0,623.211,1.59200,11022.00,10787.20,95436.8,762.069,47.8080,2.0,18.1207,1,3,13,1,73,11,0,0,12,0,0,4
2,219.403,218.933,39.7720,42.2267,81.4240,113.600,0.0,622.656,1.89133,13960.50,9669.63,95398.6,766.251,47.4320,2.0,18.4367,1,3,13,1,73,11,0,0,12,15,0,4
3,218.613,217.045,39.7760,42.0987,81.5013,113.344,0.0,622.571,1.82800,18511.20,9648.13,95360.3,766.037,47.0240,2.0,18.7513,1,3,13,1,73,11,0,0,12,30,0,4
4,217.714,216.080,39.7757,42.0686,81.4657,113.034,0.0,622.400,2.36071,26349.00,9208.32,95354.9,762.743,45.4743,2.0,19.0414,1,3,13,1,73,11,0,0,12,45,0,4


In [24]:
X = train.copy()
y = X.pop('Indoor_temperature_room')
from sklearn.model_selection import train_test_split
_, X_test, _, y_test = train_test_split( X, y, test_size=0.3, random_state=42)